# L5: Inference

<p style="background-color:#fff6e4; padding:15px; border-width:3px; border-color:#f5ecda; border-style:solid; border-radius:6px"> ⏳ <b>Note <code>(Kernel Starting)</code>:</b> This notebook takes about 30 seconds to be ready to use. You may start and watch the video while you wait.</p>

In [1]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

In [2]:
import numpy as np

from sentence_transformers import SentenceTransformer
from transformers import AutoTokenizer, \
                         DPRContextEncoder, DPRQuestionEncoder

import logging
logging.getLogger("transformers.modeling_utils").setLevel(logging.ERROR)

<p style="background-color:#fff6ff; padding:15px; border-width:3px; border-color:#efe6ef; border-style:solid; border-radius:6px"> 💻 &nbsp; <b>Access <code>requirements.txt</code> file:</b> To access <code>requirements.txt</code> for this notebook, 1) click on the <em>"File"</em> option on the top menu of the notebook and then 2) click on <em>"Open"</em>. For more help, please see the <em>"Appendix - Tips and Help"</em> Lesson.</p>

In [3]:
def cosine_similarity_matrix(features):
    
    norms = np.linalg.norm(features, axis=1, keepdims=True)
    normalized_features = features / norms
    
    similarity_matrix = np.inner(normalized_features, normalized_features)
    rounded_similarity_matrix = np.round(similarity_matrix, 4)
    
    return rounded_similarity_matrix

## Pure similarity

In [4]:
answers = [
    "What is the tallest mountain in the world?",
    "The highest mountain in the world is Mount Everest.",
    "Mount Shasta.",
    "I like my hike in the mountains.",
    "I am going to a yoga class."
]

question = answers[0]
print(question)

What is the tallest mountain in the world?


In [5]:
model = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2")
question_embedding = list(model.encode(question))

sim = []
for answer in answers:
    answer_embedding = list(model.encode(answer))
    sim.append(cosine_similarity_matrix(np.stack([question_embedding, answer_embedding]))[0,1])

print(sim)
best_inx = np.argmax(sim)
print(f"Question = {question}")
print(f"Best answer = {answers[best_inx]}")

[1.0, 0.7718, 0.3975, 0.3432, 0.086]
Question = What is the tallest mountain in the world?
Best answer = What is the tallest mountain in the world?


## Dual-Encoder inference

In [7]:
answer_tokenizer = AutoTokenizer \
                   .from_pretrained("facebook/dpr-ctx_encoder-multiset-base")
answer_encoder = DPRContextEncoder \
                   .from_pretrained("facebook/dpr-ctx_encoder-multiset-base")

question_tokenizer = AutoTokenizer \
                   .from_pretrained("facebook/dpr-question_encoder-multiset-base")
question_encoder = DPRQuestionEncoder \
                   .from_pretrained("facebook/dpr-question_encoder-multiset-base")

tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/492 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/438M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/493 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/438M [00:00<?, ?B/s]

In [8]:
# Compute the question embeddings
question_tokens = question_tokenizer(question, return_tensors="pt")["input_ids"]
question_embedding = question_encoder(question_tokens).pooler_output.flatten().tolist()

print(question_embedding[:10], len(question_embedding))

[0.07758200913667679, 0.251725435256958, 0.18663956224918365, 0.22120118141174316, 0.026415381580591202, -0.15785622596740723, 0.32760292291641235, 0.2673285901546478, -0.08503079414367676, 0.1292949914932251] 768


In [9]:
sim = []
for answer in answers:
    answer_tokens = answer_tokenizer(answer, return_tensors="pt")["input_ids"]
    answer_embedding = answer_encoder(answer_tokens).pooler_output.flatten().tolist() 
    sim.append(cosine_similarity_matrix(np.stack([question_embedding, answer_embedding]))[0,1])

print(sim)
best_inx = np.argmax(sim)
print(f"Question = {question}")
print(f"Best answer = {answers[best_inx]}")

[0.6253, 0.7346, 0.5965, 0.3168, 0.2565]
Question = What is the tallest mountain in the world?
Best answer = The highest mountain in the world is Mount Everest.
